In [12]:
import os
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()

True

In [13]:
provinces = pd.read_csv("../dataset/provinces.csv")
provinces.head()

,code,name
0,11,ACEH
1,12,SUMATERA UTARA
2,13,SUMATERA BARAT
3,14,RIAU
4,15,JAMBI


In [14]:
regencies = pd.read_csv("../dataset/regencies.csv")
regencies.head()

,code,province_code,name
0,11.01,11,KABUPATEN ACEH SELATAN
1,11.02,11,KABUPATEN ACEH TENGGARA
2,11.03,11,KABUPATEN ACEH TIMUR
3,11.04,11,KABUPATEN ACEH TENGAH
4,11.05,11,KABUPATEN ACEH BARAT


In [15]:
districts = pd.read_csv("../dataset/districts.csv")
districts.head()

,code,regency_code,name
0,11.01.01,11.01,Bakongan
1,11.01.02,11.01,Kluet Utara
2,11.01.03,11.01,Kluet Selatan
3,11.01.04,11.01,Labuhanhaji
4,11.01.05,11.01,Meukek


In [16]:
villages = pd.read_csv("../dataset/villages.csv")
villages.head()

,code,district_code,name
0,11.01.01.2001,11.01.01,Keude Bakongan
1,11.01.01.2002,11.01.01,Ujong Mangki
2,11.01.01.2003,11.01.01,Ujong Padang
3,11.01.01.2004,11.01.01,Gampong Drien
4,11.01.01.2015,11.01.01,Darul Ikhsan


In [17]:
conn = mysql.connector.connect(
    host= os.getenv("MYSQL_HOST"),
    user= os.getenv("MYSQL_USER"),
    password= os.getenv("MYSQL_PASSWORD"),
    database= os.getenv("MYSQL_DATABASE")
)

cursor = conn.cursor()
cursor.execute("SELECT DATABASE()")
db_name = cursor.fetchone()[0]
print(f"Berhasil terhubung ke database: {db_name}")

Berhasil terhubung ke database: nusantara-kita


In [18]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS provinces (
    code VARCHAR(2) PRIMARY KEY,
    name VARCHAR(255) NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS regencies (
    code VARCHAR(5) PRIMARY KEY,
    province_code VARCHAR(2),
    name VARCHAR(255) NOT NULL,
    FOREIGN KEY (province_code) REFERENCES provinces(code)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS districts (
    code VARCHAR(8) PRIMARY KEY,
    regency_code VARCHAR(5),
    name VARCHAR(255) NOT NULL,
    FOREIGN KEY (regency_code) REFERENCES regencies(code)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS villages (
    code VARCHAR(13) PRIMARY KEY,
    district_code VARCHAR(8),
    name VARCHAR(255) NOT NULL,
    FOREIGN KEY (district_code) REFERENCES districts(code)
)
""")

conn.commit()

def show_query_results(cursor, query):
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

table_queries = [
        "DESCRIBE provinces",
        "DESCRIBE regencies",
        "DESCRIBE districts",
        "DESCRIBE villages"
    ]
    
for query in table_queries:
    print(f"\nStruktur tabel untuk {query.split()[1]}:")
    show_query_results(cursor, query)


Struktur tabel untuk provinces:
('code', 'varchar(2)', 'NO', 'PRI', None, '')
('name', 'varchar(255)', 'NO', '', None, '')

Struktur tabel untuk regencies:
('code', 'varchar(5)', 'NO', 'PRI', None, '')
('province_code', 'varchar(2)', 'YES', 'MUL', None, '')
('name', 'varchar(255)', 'NO', '', None, '')

Struktur tabel untuk districts:
('code', 'varchar(8)', 'NO', 'PRI', None, '')
('regency_code', 'varchar(5)', 'YES', 'MUL', None, '')
('name', 'varchar(255)', 'NO', '', None, '')

Struktur tabel untuk villages:
('code', 'varchar(13)', 'NO', 'PRI', None, '')
('district_code', 'varchar(8)', 'YES', 'MUL', None, '')
('name', 'varchar(255)', 'NO', '', None, '')


In [19]:
def insert_data(table_name, data_frame, cursor):
    rows_inserted = 0
    for index, row in data_frame.iterrows():
        columns = ", ".join(data_frame.columns)
        placeholders = ", ".join(["%s"] * len(row))
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(sql, tuple(row))
        rows_inserted += 1
    print(f"Data berhasil di-insert ke tabel {table_name}: {rows_inserted} baris")

insert_data("provinces", provinces, cursor)
insert_data("regencies", regencies, cursor)
insert_data("districts", districts, cursor)
insert_data("villages", villages, cursor)

conn.commit()

Data berhasil di-insert ke tabel provinces: 38 baris
Data berhasil di-insert ke tabel regencies: 514 baris
Data berhasil di-insert ke tabel districts: 7277 baris
Data berhasil di-insert ke tabel villages: 83763 baris


In [20]:
DATABASE_URI = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URI)

def read_from_mysql(table_name):
    query = f"SELECT * FROM {table_name}"
    return pd.read_sql(query, engine)

mysql_provinces = read_from_mysql("provinces")
mysql_provinces.head()

,code,name
0,11,ACEH
1,12,SUMATERA UTARA
2,13,SUMATERA BARAT
3,14,RIAU
4,15,JAMBI


In [21]:
mysql_regencies = read_from_mysql("regencies")
mysql_regencies.head()

,code,province_code,name
0,11.01,11,KABUPATEN ACEH SELATAN
1,11.02,11,KABUPATEN ACEH TENGGARA
2,11.03,11,KABUPATEN ACEH TIMUR
3,11.04,11,KABUPATEN ACEH TENGAH
4,11.05,11,KABUPATEN ACEH BARAT


In [22]:
mysql_districts = read_from_mysql("districts")
mysql_districts.head()

,code,regency_code,name
0,11.01.01,11.01,Bakongan
1,11.01.02,11.01,Kluet Utara
2,11.01.03,11.01,Kluet Selatan
3,11.01.04,11.01,Labuhanhaji
4,11.01.05,11.01,Meukek


In [23]:
mysql_villages = read_from_mysql("villages")
mysql_villages.head()

,code,district_code,name
0,11.01.01.2001,11.01.01,Keude Bakongan
1,11.01.01.2002,11.01.01,Ujong Mangki
2,11.01.01.2003,11.01.01,Ujong Padang
3,11.01.01.2004,11.01.01,Gampong Drien
4,11.01.01.2015,11.01.01,Darul Ikhsan


In [24]:
def compare_dataframes(df1, df2):
    df1 = df1.sort_values(by=list(df1.columns)).reset_index(drop=True)
    df2 = df2.sort_values(by=list(df2.columns)).reset_index(drop=True)
    return df1.equals(df2)

provinces_match = compare_dataframes(provinces, mysql_provinces)
regencies_match = compare_dataframes(regencies, mysql_regencies)
districts_match = compare_dataframes(districts, mysql_districts)
villages_match = compare_dataframes(villages, mysql_villages)

print(f"Provinces data match: {provinces_match}")
print(f"Regencies data match: {regencies_match}")
print(f"Districts data match: {districts_match}")
print(f"Villages data match: {villages_match}")

Provinces data match: False
Regencies data match: False
Districts data match: False
Villages data match: True


In [25]:
print("DATA CSV")
print(provinces.dtypes)
print("DATA MYSQL")
print(mysql_provinces.dtypes)

DATA CSV
code     int64
name    object
dtype: object
DATA MYSQL
code    object
name    object
dtype: object


In [26]:
print("DATA CSV")
print(regencies.dtypes)
print("DATA MYSQL")
print(mysql_regencies.dtypes)

DATA CSV
code             float64
province_code      int64
name              object
dtype: object
DATA MYSQL
code             object
province_code    object
name             object
dtype: object


In [27]:
print("DATA CSV")
print(districts.dtypes)
print("DATA MYSQL")
print(mysql_districts.dtypes)

DATA CSV
code             object
regency_code    float64
name             object
dtype: object
DATA MYSQL
code            object
regency_code    object
name            object
dtype: object


In [28]:
print("DATA CSV")
print(villages.dtypes)
print("DATA MYSQL")
print(mysql_villages.dtypes)

DATA CSV
code             object
district_code    object
name             object
dtype: object
DATA MYSQL
code             object
district_code    object
name             object
dtype: object


In [29]:
def find_differences(df1, df2):
    df1 = df1.sort_values(by=list(df1.columns)).reset_index(drop=True)
    df2 = df2.sort_values(by=list(df2.columns)).reset_index(drop=True)
    
    differences = df1.compare(df2)
    return differences

provinces_differences = find_differences(provinces, mysql_provinces)
regencies_differences = find_differences(regencies, mysql_regencies)
districts_differences = find_differences(districts, mysql_districts)
villages_differences = find_differences(villages, mysql_villages)

print("Provinces differences:\n", provinces_differences, "\n")
print("Regencies differences:\n", regencies_differences, "\n")
print("Districts differences:\n", districts_differences, "\n")
print("Villages differences:\n", villages_differences, "\n")

Provinces differences:
    code      
   self other
0    11    11
1    12    12
2    13    13
3    14    14
4    15    15
5    16    16
6    17    17
7    18    18
8    19    19
9    21    21
10   31    31
11   32    32
12   33    33
13   34    34
14   35    35
15   36    36
16   51    51
17   52    52
18   53    53
19   61    61
20   62    62
21   63    63
22   64    64
23   65    65
24   71    71
25   72    72
26   73    73
27   74    74
28   75    75
29   76    76
30   81    81
31   82    82
32   91    91
33   92    92
34   93    93
35   94    94
36   95    95
37   96    96 

Regencies differences:
       code        province_code      
      self  other          self other
0    11.01  11.01            11    11
1    11.02  11.02            11    11
2    11.03  11.03            11    11
3    11.04  11.04            11    11
4    11.05  11.05            11    11
..     ...    ...           ...   ...
509  96.02  96.02            96    96
510  96.03  96.03            96    96
511  96.04

In [30]:
provinces["code"] = provinces["code"].astype(str)
regencies["code"] = regencies["code"].astype(str)
regencies["province_code"] = regencies["province_code"].astype(str)
districts["code"] = districts["code"].astype(str)
districts["regency_code"] = districts["regency_code"].astype(str)
villages["code"] = villages["code"].astype(str)
villages["district_code"] = villages["district_code"].astype(str)
print("Tipe data columns telah berhasil dikonversi")

Tipe data columns telah berhasil dikonversi
